In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
import time
from time import sleep

In [72]:
air_df = pd.read_csv('./data/purpleair.csv')
air_df.head(1)

,id,parent,lat,lon,name,location_type,pm_2.5,temp_f,temp_c,humidity,...,last_update_check,created,uptime,is_owner,10min_avg,30min_avg,1hour_avg,6hour_avg,1day_avg,1week_avg
0,26000.0,25999.0,30.053808,-95.494643,Villages of Bridgestone AQI B,NaN,0.42,NaN,NaN,NaN,...,NaN,NaN,NaN,False,1.13,2.38,5.85,16.33,13.94,12.16


#### Need to update the following work for the purple air df (air_df):

In [70]:
def lat_lon_query(lat, lon, cen_yr):

    url_fips = f'https://geo.fcc.gov/api/census/block/find?latitude={lat}&longitude={lon}&censusYear={cen_yr}&showall=true&format=json'
    response = requests.get(url_fips)

    if response.status_code == 200:
        aq_fips = response.json()
        aq_fips_code = aq_fips['Block']['FIPS']
        print(aq_fips_code)
    else:
        print('Error!')

    time.sleep(1)

    return aq_fips_code

In [71]:
aq_df.apply(lambda row: lat_lon_query(row['Latitude'], row['Longitude'], 2010), axis=1)

060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
060372060102002
06037206

0      060372060102002
1      060372060102002
2      060372060102002
3      060372060102002
4      060372060102002
            ...       
497    060379800281012
498    060379800281012
499    060379800281012
500    060379800281012
501    060379800281012
Length: 502, dtype: object

In [55]:
walk_df = pd.read_csv('./LA_data/walkability_LA.csv')
walk_df.head(1)

,Unnamed: 0,OBJECTID,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,...,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area
0,31470,31471,6.037187e+10,6.037187e+10,6,37,187200,2,348.0,"Los Angeles-Long Beach, CA",...,0.309765,0.000133,0.17882,15,7,16,13,13.333333,3796.740538,552006.8969


In [56]:
walk_df['GEOID10'][0]

# convert to int
# convert to string
# add 0 for to the front - all FIPs codes should be 12 digits

60371872002.0

In [ ]:
for item in walk_df['STATEFP']:
    string_item = str(item).zfill(2)
    print(string_item)